**Содержание**
* [Название проекта.](#Name)
    -  [Цель исследования.](#purpose)
    -  [Задачи исследования.](#goal)
    -  [Исходные данные.](#data)
    
 [Обзор данных.](#overview)
 
 [Предобработка данных.](#preprocessing)
    - [2.1 Обработка пропусков.](#skip)
    - [2.2 Замена типа данных.](#type)
    - [2.3 Обработка дупликатов.](#duplicates)
    - [2.5 Категоризация данных.](#category)
    
 [Проверка гипотез.](#hypothesis)
 
 [Общий вывод.](#conclusion)

<a id='Name'></a>
# 1. Исследование надёжности заёмщиков 

<a id='purpose'></a>
**Цель исследования** — разобраться:
1. Влияет ли количество детей и семейное положение клиента на факт погашения кредита в срок?
2. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?
3. Как разные цели кредита влияют на его возврат в срок?

<a id='goal'></a>
**Задачи исследования:**
1. Провести анализ общей информации.
2. Выполнить предобработку данных.
3. Проверить гипотезы.
4. Составить выводы на основе полученных результатов.

<a id='data'></a>
**Исходные данные:**
Статистика о платёжеспособности клиентов от кредитного отдела банка.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

<a id='overview'></a>
# 2. Изучение файлов с данными, получение общей информации, загрузка библиотек.

In [1]:
#импортируем нужные библиотеки
import pandas as pd 
from pymystem3 import Mystem
m = Mystem()

df = pd.read_csv('/datasets/data.csv') 
df.info() 
display(df.head(15)) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод**

Изучив общую информацию видим несколько проблем с данными: в столбце days_employed и total_income имеются пропущенные значения. Также days_empoloyed имеет отрицательные значения и тип float, а должен быть int, так как в нем записан трудовой стаж в целых днях. В столбец total_income также имеет тип float, для удобства вычеслений можно привести его к целочисленному типу.
Отобразив первые 15 строк таблицы видим, что столбец days_employed помимо пропусков имеет отрицательные значения, а столбец education содержит дупликаты.


<a id='preprocessing'></a>
##  Предобработка данных

<a id='skip'></a>
### Обработка пропусков

In [2]:
#методом value_counts() посмотрим уникальные значения в столбцах children и education.

print(df['children'].value_counts()) 
print(df['education'].value_counts()) 
#выдвинул гипотезу, что в случае с 20 детьми был поставлен лишний 0 и заменил значение на 2.
df['children'] = df['children'].replace(20, 2) 

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64
среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64


**Вывод**

В столбце children видим отрицательные значения и одно сильно выделяющееся: 20. Безусловно это ошибки в данных, которые необходимо обработать. В education видим дупликаты из-за разных регистров, это также следует исправить.

In [3]:
#функция, которая умножает отрицательные значения на -1, если значение положительное, то функция возвращает это значение
def to_pos(value): 
    if value < 0:
        value *= -1
        return value
    else:
        return value
#применяем функцию к нужным столбцам
df['days_employed'] = df['days_employed'].apply(to_pos) 
df['children'] = df['children'].apply(to_pos)


In [4]:
df['days_employed'] = df['days_employed'].fillna(df.groupby('income_type')['days_employed'].transform('median'))
df['total_income'] = df['total_income'].fillna(df.groupby('income_type')['days_employed'].transform('median'))
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     21525 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      21525 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

В слобцах days_employed и total_income были обнаружены пропуски NaN. Причиной этому могут быть ошибки при вводе данных или ошибки при выгрузке. NaN принадлежит типу float, следовательно с ним можно проводить математические операции, но для получения более точных результатов лучше заполним эти пропуски используя метод .fillna(), позволяющий заполнить значения NaN в определенных столбцах датафрейма. Ежемесячный доход и трудовой стаж - количественные переменные, поэтому заполняем пропуски в строках медианным значением в соответствии с типом занятости человека. Медиана выбрана, тк среднее значение может некорректно охарактеризовать данные: некоторые значения могут существенно выделяться среди большинства.

<a id='type'></a>
### Замена типа данных

In [5]:
#заменим типы данных в столбцах с float на int методом .astype()
df['days_employed'] = df['days_employed'].astype('int') 
df['total_income'] = df['total_income'].astype('int')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

Трудовой стаж и заработок приведены к целочисленным значениям методом .astype(), который позволяет изменять тип данных целых столбцов. Это упростит расчеты в дальнейшем.

<a id='duplicates'></a>
### Обработка дубликатов

In [6]:
#Приведем строки столбца education к нижнему регистру
df['education'] = df['education'].str.lower() 
print(df['education'].value_counts())

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64


In [7]:
#избавимся от явных дупликатов

df = df.drop_duplicates().reset_index(drop=True)
print('Количество явных дубликатов:', df.duplicated().sum())

Количество явных дубликатов: 0


**Вывод**

В исходном датафрейме имеется некоторое количество дубликатов. Причиной их появления могут быть ошибки при занесении информации в таблицу.
Некоторые строки столбца education отличались регистром, приведя их нижнему регистру методом str.lower() мы избавились от неявных дубликатов.
Также методом .drop_duplicates() избавиляемся от явных дубликатов таблицы. Для проверки выведем их количество после применения данного метода.

<a id='category'></a>
###  Категоризация данных

In [8]:
#по рекомандации преподавателя написал функцию, которая лемматизирует и сразу делает категоризацию, 
#применил ее указав параметр axis=1, чтобы на вход в функцию отправлялись строки
def create_category_purpose(row):      
    lem_purpose = m.lemmatize(row['purpose'])
                                                             
    if 'автомобиль' in lem_purpose:                          
        return 'автомобиль'
    if ('жилье' in lem_purpose) or ('недвижимость' in lem_purpose):
        return 'недвижимость'
    if 'образование' in lem_purpose:
        return 'образование'
    if 'свадьба' in lem_purpose:
        return 'свадьба'
    return 'другое'

#не уверен на счет правильности категоризцаии заработка, показалось логичным поделилить их на 4 категории с шагом 50000

def create_income_category(row):
    income = row['total_income']
    if income <= 50000:
        return 'низкий'
    if income > 50000 and income <= 100000:
        return 'средний'
    if income > 100000 and income <= 150000:
        return 'высокий'
    if income > 150000:
        return 'очень высокий'
    
df['purpose_category'] = df.apply(create_category_purpose, axis=1)
df['income_category'] = df.apply(create_income_category, axis=1)
display(df.head(20))

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_category,income_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,недвижимость,очень высокий
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,высокий
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,недвижимость,высокий
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,очень высокий
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,свадьба,очень высокий
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,недвижимость,очень высокий
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,недвижимость,очень высокий
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование,высокий
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,свадьба,средний
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,недвижимость,высокий


**Вывод**

Благодаря двум функциям создаем два столбца таблицы, в которые будут занесены категории целей получения кредита и категории заработка соответственно. Новые столбцы помогут нам в ответе на вопросы, поставленные в целях данного иследования.

<a id='hypothesis'></a>
##  Проверка гипотез.

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [9]:
pivot_table_children = df.pivot_table(index='children', columns= 'debt', values='days_employed', aggfunc='count')
pivot_table_children['percent'] = pivot_table_children[1]/(pivot_table_children[0] + pivot_table_children[1]) * 100
display(pivot_table_children.sort_values(by='percent', ascending=False))

debt,0,1,percent
children,,,
4,37.0,4.0,9.756098
2,1926.0,202.0,9.492481
1,4410.0,445.0,9.165808
3,303.0,27.0,8.181818
0,13028.0,1063.0,7.543822
5,9.0,NaN,NaN


**Вывод**

По таблице видно, что чем больше детей, тем более высока вероятность задержки выплаты кредита. Хотя это правило не работает для людей с 3 детьми, тк вероятность задержки у людей с одним/двумя детьми выше. Значение NaN показывает, что должников среди людей с 5 детьми нет.


- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [10]:
pivot_table_family = df.pivot_table(index='family_status', columns= 'debt', values='days_employed', aggfunc='count')
pivot_table_family['percent'] = pivot_table_family[1]/(pivot_table_family[0] + pivot_table_family[1]) * 100
display(pivot_table_family.sort_values(by='percent', ascending=False))

debt,0,1,percent
family_status,,,
Не женат / не замужем,2536,274,9.750890
гражданский брак,3763,388,9.347145
женат / замужем,11408,931,7.545182
в разводе,1110,85,7.112971
вдовец / вдова,896,63,6.569343


**Вывод**

Люди, не состоящие в браке имеют большую вероятность невыплаты кредита в срок. Вдовы и вдовцы, а также люди в разводе имеют наименьшую вероятность просрочить выплату.
Я бы предположил, что люди состоящие в браке помогают друг другу в выплате кредита, поэтому они имеют меньший процент вероятности просрочки кредита. Но в таком случае непонятно, почему вдовы/вдовцы и люди в разводе имеют еще более меньший процент.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [11]:
pivot_table_income = df.pivot_table(index='income_category', columns= 'debt', values='days_employed', aggfunc='count')
pivot_table_income['percent'] = pivot_table_income[1]/(pivot_table_income[0] + pivot_table_income[1]) * 100
display(pivot_table_income.sort_values(by='percent', ascending=False))

debt,0,1,percent
income_category,,,
высокий,5213,491,8.607994
средний,3760,331,8.090931
очень высокий,8809,761,7.951933
низкий,1931,158,7.563427


**Вывод**

Люди, имеющие низкий зароботок имеют самый маленький процент по невыплатам. Люди с очень высоким заработком на третьем месте. Зависимости нет.

- Как разные цели кредита влияют на его возврат в срок?

In [12]:
pivot_table_purpose_category = df.pivot_table(index='purpose_category', columns= 'debt', values='days_employed', aggfunc='count')
pivot_table_purpose_category['percent'] = pivot_table_purpose_category[1]/(pivot_table_purpose_category[0] + pivot_table_purpose_category[1]) * 100
display(pivot_table_purpose_category.sort_values(by='percent', ascending=False))

debt,0,1,percent
purpose_category,,,
автомобиль,3903,403,9.359034
образование,3643,370,9.220035
свадьба,2138,186,8.003442
недвижимость,10029,782,7.233373


**Вывод**

Люди, целью которых является покупка автомобиля или получение образования имеют больший шанс не выплатить кредит вовремя. 

<a id='conclusion'></a>
##  Общий вывод

В результате исследования мы смогли получить ответы на все интересующие нас вопросы, а именно: выяснили как влияет количество детей и семейное положение, разные цели и уровень дохода клиента на факт погашения кредита в срок. Чтобы получить корректные ответы была проведена предобработка исходных данных: замена типов данных у нужных нам столбцов, заполнение пропусков и удаление дупликатов. Все эти ошибки могли повлиять на правильность выводов в каждом пункте проекта. Для более лучшего восприятия результатов построены таблицы, изучив которые можно составить общую картину клиентов. Вероятнее всего у людей могут возникнуть проблемы с выплатой в случае, если: они не состоят в браке, у них имеются дети и целью взятия кредита являются покупка автомобиля, либо получения образования.

